# Training a Contrail Classifier
The goal of this project is to train a machine learning model that can accurately classify images of the sky as containing contrails.
To build the model, we have obtained cloud data from four sources:
1. [Cirrus Cumulus Stratus Nimbus (CCSN) Database](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/CADDPD), this 
2. [Singapore Data Swimcat](https://ieeexplore.ieee.org/abstract/document/7350833)
3. [CLASA](https://github.com/CLASA/Contrail-Machine-Vision), [Official Website](https://clasa.github.io/) a proposed solution to the NASA Clouds vs Contrails challenge.
4. [Google Cloud Project](https://arxiv.org/abs/2304.02122)

Potential applications are noted below:

Potential Applications
* Climate Studies: Contrails can have a significant impact on the Earth's atmosphere and climate. They can reflect sunlight back into space, contributing to global cooling, but they can also trap heat within the Earth's atmosphere, contributing to global warming. Therefore, a machine learning model trained to detect and monitor contrails could provide important data for climate researchers.

* Air Traffic Control: A model trained to identify contrails could be useful in tracking aircraft routes and densities, particularly in areas with less developed radar infrastructure.

* Aerospace and Defense: This model could be used for aerospace and defense purposes. For instance, detecting contrails could help in tracking and identifying stealth, unauthorized, or unrecognized flights, which can be important in maintaining airspace security.

# Preprocessing Script
Preprocessing is a crucial step in the machine learning pipeline because the quality and quantity of the data that you feed into your model will directly determine how well it can learn. Here are some reasons how we could preprocess image data:
* Labeling: These images are not all labeled, and images from different datasets. The purpose of labeling is to homogenize the data so that each image is labeled in the same manner.

* Image Resizing: In real-world scenarios, images can come in different sizes and aspect ratios. However, many computer vision models (like Convolutional Neural Networks) require images to be of a uniform size. Therefore, images often need to be resized to fit the requirements of the model.

* Normalization: Image pixel intensities can range from 0 to 255. Normalizing these pixel intensities to a smaller range, often between 0 and 1 or -1 and 1, can help the model learn more effectively. This is because smaller, centered values are easier for the model's weight initialization and optimization process. Scaling the pixel values of the images to a small range like 0-1 or -1 to 1 can help the model converge faster during training. The 'Rescaling' layer in TensorFlow can be used for this purpose.

* Data Augmentation: Image datasets can be augmented by applying random transformations like rotation, scaling, translation, flip etc. This can help increase the amount of training data and make the model more robust to variations in the input data that it hasn't seen before. This can help the model generalize better to new data. TensorFlow provides tools for data augmentation in the 'tf.keras.layers.experimental.preprocessing' module.

* Dealing with Color Channels: Some models might require grayscale images, while others might require color images. Depending on the model, you might need to convert images from color to grayscale, or vice versa. Depending on your data, you might find that transforming the color space of your images (from RGB to HSV, Lab, YUV, etc.) could improve your model's ability to detect features.

* Feature Extraction: In some cases, it might be beneficial to manually extract features from the images, such as edges, corners, and other local features. These can be used as inputs to the machine learning model. For contrails detection, specific filters that are sensitive to the features of contrails could be used. This might require some research and experimentation.

* Dimensionality Reduction: Images are high-dimensional data, and it may be beneficial to reduce their dimensionality. This can be done through techniques like Principal Component Analysis (PCA) or autoencoders, which can make the model more efficient without losing too much information.

* Balancing Classes: If the numbers of contrail and no-contrail images are not roughly equal, the model might become biased towards the more common class. Solutions include oversampling the minority class, undersampling the majority class, or using a combination of both.

These preprocessing steps help to make the image data more suitable for computer vision models and can lead to better performance.

## Loading the Data and Labeling

In [96]:
import os
import io
from PIL import Image, ImageOps
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from collections import Counter

In [97]:
def image_loader(image_dir, dictionary, filetype):
    # List to hold all image data
    images = []
    # List to hold image classifications (1,0)
    classes = []
    # Iterate over each image in the directory
    for folder in os.listdir(image_dir):
        # Only open files with .jpg extension (or add your extension)
        for filename in os.listdir(image_dir+'/'+folder):
            if filename.endswith(filetype):
                # Open each image file
                img = Image.open(os.path.join(image_dir+'/'+folder, filename))
                # If the image has an alpha channel, it needs to be converted to RGB
                #if filetype == ".png":
                    # Convert the image into RGB and save it as a JPEG so its homogenized
                #    img = img.convert('RGB')
                #    
                # Append the image data to your list
                images.append(img)
                classes.append(dictionary[folder])
    # Now the 'images' list contains all the images im the image_dir as PIL Image objects, with the labels in the 'classes' list
    return(images, classes)

In [98]:
image_dir = "../data/CCSN_v2"
# Dictionary to translate scientific names into contrail labels
ccsn_dictionary = {
    'Ct':1,
    'Ac':0, 'Sc':0, 'Ns':0, 'Cu':0, 'Ci':0, 'Cc':0, 'Cb':0, 'As':0, 'Cs':0, 'St':0
}
images_ccsn, classes_ccsn = image_loader(image_dir, ccsn_dictionary, ".jpg")
print(f'From CCSN we extract {len(images_ccsn)} images and {len(classes_ccsn)} classes')

From CCSN we extract 2543 images and 2543 classes


In [99]:
print(classes_ccsn)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [100]:
image_dir = "../data/CLASA"
# Dictionary to translate names into contrail labels
ccsn_dictionary = {
    'Contrail':1,
    'Cirrus':0
}
images_clasa, classes_clasa = image_loader(image_dir, ccsn_dictionary, ".jpg")
print(f'From CLASA we extract {len(images_clasa)} images and {len(classes_clasa)} classes')

From CLASA we extract 454 images and 454 classes


In [101]:
image_dir = "../data/Singapore Data Swimcat"
# Dictionary to SWIMCAT labels into contrail labels
ccsn_dictionary = {
    'A-sky':0,
    'B-pattern':0,
    'C-thick-dark':0,
    'D-thick-white':0,
    'E-veil':0
}
images_singapore, classes_singapore = image_loader(image_dir, ccsn_dictionary, ".png")
print(f'From Singapore we extract {len(images_singapore)} images and {len(classes_singapore)} classes')

From Singapore we extract 784 images and 784 classes


In [102]:
# Connect to the Google Cloud api, so that we pull the data only when we need it

In [103]:
# Merge all the folders into two lists: one containing images, and the other has the labels
images_all = images_ccsn + images_clasa + images_singapore
classes_all = classes_ccsn + classes_clasa + classes_singapore
print(images_all)
print(classes_all)
print(f'In total we have extracted {len(images_all)} images and {len(classes_all)} classes')

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=400x400 at 0x23D8A48B2B0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=400x400 at 0x23B8C1BE590>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=400x400 at 0x23D8A4886D0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=400x400 at 0x23DFF71AD70>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=400x400 at 0x23DFF71AFB0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=400x400 at 0x23DFF71ADA0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=400x400 at 0x23DFF71B190>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=400x400 at 0x23DFF719F30>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=400x400 at 0x23DFF718D60>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=400x400 at 0x23DFF71A290>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=400x400 at 0x23DFF7184F0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=400x400 at 0x23DFF719540>, <PIL.JpegImageP

In [104]:
# Do some feature analysis to see the distribution of response variable
print(f'Proportion of contrails in the total sample is {round(np.mean(classes_all),2)}')


Proportion of contrails in the total sample is 0.15


In [105]:
# Do some feature analysis to see the distribution of input iamges
def image_dimension(images):
    # Get dimensions of images
    dimensions = [img.size for img in images]

    # Split dimensions into two lists: width and height
    widths = [dim[0] for dim in dimensions]
    heights = [dim[1] for dim in dimensions]

    # Create subplots: 2 rows, 1 column
    fig = make_subplots(rows=3, cols=1)

    # Add histogram for widths to the first subplot
    fig.add_trace(
        go.Histogram(x=widths, name='widths', opacity=0.75),
        row=1, col=1
    )

    # Add histogram for heights to the second subplot
    fig.add_trace(
        go.Histogram(x=heights, name='heights', opacity=0.75),
        row=2, col=1
    )

    # Add histogram for heights to the second subplot
    fig.add_trace(
        go.Histogram(x=np.array(widths)/np.array(heights), name='aspect_ratio', opacity=0.75),
        row=3, col=1
    )

    # Update xaxis titles
    fig.update_xaxes(title_text='Widths', row=1, col=1)
    fig.update_xaxes(title_text='Heights', row=2, col=1)
    fig.update_xaxes(title_text='Aspect Ratio', row=3, col=1)

    # Update yaxis titles
    fig.update_yaxes(title_text='Count', row=1, col=1)
    fig.update_yaxes(title_text='Count', row=2, col=1)
    fig.update_yaxes(title_text='Count', row=3, col=1)

    # Update layout to show subplots
    fig.update_layout(
        title_text='Distribution of Image Widths and Heights', # title of plot
        height=600, # height of plot in pixels
        width=900, # width of plot in pixels
    )

    fig.show()
image_dimension(images_all)

In [106]:
# Find the most common pixel size so that we can make squares out of that size
# Get dimensions of images
dimensions = [img.size for img in images_all]

# Split dimensions into two lists: width and height
widths = [dim[0] for dim in dimensions]
heights = [dim[1] for dim in dimensions]
most_common_width = Counter(widths).most_common(1)[0][0]
most_common_height = Counter(heights).most_common(1)[0][0]
print(f'From all images, the most_common width is {most_common_width} px and the most common height is {most_common_height} px')
print(f'From all images, the smallest width is {np.min(widths)} px and the maximum width is {np.max(widths)} px')
print(f'From all images, the smallest height is {np.min(heights)} px and the maximum width is {np.max(heights)} px')

From all images, the most_common width is 400 px and the most common height is 400 px
From all images, the smallest width is 120 px and the maximum width is 7437 px
From all images, the smallest height is 95 px and the maximum width is 5315 px


In [107]:
# Make the images square and do other transformations
pixels = 400
images_all_squared = []
classes_all_squared = []
i = 0
for img in images_all:
    width, height = img.size
    if width < pixels or height < pixels:
        # Calculate padding
        width_padding = pixels - width
        height_padding = pixels - height
        # Apply padding with a grey background
        images_all_squared.append(ImageOps.pad(img, (pixels,pixels), color=125))
    elif width > 400 or height > 400:
        # Try both shrinking the image and cropping the image to create syntetic samples
        # Image.LANCZOS applies a high-quality downsampling filter
        images_all_squared.append(img.resize((pixels, pixels), Image.LANCZOS))
        # Returns a resized and cropped version of the image, cropped to the requested aspect ratio and size.
        images_all_squared.append(ImageOps.fit(img, (pixels, pixels)))
        # Because we are expanding the dataset, make sure to add the extra class labels
        classes_all_squared.append(classes_all[i])
    else:
        images_all_squared.append(img)
    classes_all_squared.append(classes_all[i])
    i += 1

print(f'In total we have turned {len(images_all)} raw images into {len(images_all_squared)} square images with {pixels} px sides')

In total we have turned 3781 raw images into 3964 square images with 400 px sides


In [108]:
image_dimension(images_all_squared)

## Train test split

In [109]:
from sklearn.model_selection import train_test_split

# Assume X is your array of features and y are the labels
X_train, X_test, y_train, y_test = train_test_split(images_all_squared, classes_all_squared, test_size=0.2, random_state=42)
print(f'X_train is {len(X_train)} images long')
print(f'X_test is {len(X_test)} images long')
print(f'y_train is {len(y_train)} labels long')
print(f'y_test is {len(y_test)} labels long')

X_train is 3171 images long
X_test is 793 images long
y_train is 3171 labels long
y_test is 793 labels long


## Normalization

## Dealing with Color Channels

## Feature Extraction

## Dimensionality Reduction

## Image Resizing

In [110]:
from keras.preprocessing.image import ImageDataGenerator

In [111]:
random_datagen = ImageDataGenerator(
    rescale = 1/255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
    )
# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
random_datagen.fit(X_train)
         

c:\Users\jhqui\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\preprocessing\image.py:2085: FutureWarning:

The input object of type 'Image' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Image', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.



ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3171,) + inhomogeneous part.

## Create Model

In [ ]:
from keras.applications import ResNet50
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam

In [ ]:
# Import the pre-built ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False)

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# Add a logistic layer for binary classification
predictions = Dense(1, activation='sigmoid')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

# Compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

94765736/94765736 [==============================] - 7s 0us/step


# Balancing the Data

In [ ]:
from sklearn.utils import class_weight
import numpy as np

# Calculate class weights
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)

# Convert class_weights to a dictionary to include it in model.fit()
class_weights = dict(enumerate(class_weights))

NameError: name 'y_train' is not defined

# Fitting the Model

In [ ]:
# fits the model on batches with real-time data augmentation:
# Pass class_weights to the fit function
model.fit(random_datagen.flow(X_train, y_train, batch_size=32, subset='training'),
         validation_data=random_datagen.flow(X_train, y_train, batch_size=8, subset='validation'),
         steps_per_epoch=len(X_train) / 32, epochs=32, class_weights = class_weights)

# Testing the Model
When dealing with imbalanced classes, traditional metrics like accuracy can be misleading. For a task where avoiding false positives (i.e., the model predicting a positive class when it's actually negative) is important, you might want to consider the following metrics:

* Precision: Precision is the ratio of true positives (TP) to the sum of true positives and false positives (FP). Precision is directly concerned with minimizing false positive predictions. Precision = TP / (TP + FP)

* F1 Score: The F1 score is the harmonic mean of precision and recall. While it doesn't directly focus on false positives, it provides a balance between precision and recall. This can be useful if both false positives and false negatives are of concern.

* Area Under the Precision-Recall Curve (AUPRC): In an imbalanced classification problem, AUPRC can be a better metric than traditional ones. It calculates the area under the curve formed by plotting recall (x-axis) against precision (y-axis) at various threshold settings. The closer this area is to 1, the better the model is at distinguishing between the positive and negative classes.

For the cost function in the training phase of a neural network, the standard is cross-entropy loss. When dealing with imbalanced classes, one way to handle this is by applying class weights to the loss function, which assigns a higher penalty for misclassifying the minority class.